In [ ]:
import datetime
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.style.use('seaborn-bright')
plt.rcParams['figure.figsize'] = [15, 9]
plt.rcParams['font.size'] = 12

pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

In [ ]:
balanced = "RefBldgSmallOfficeNew2004_USA_KY_Cincinnati-Northern.Kentucky.AP.724210_TMY.csv"
imbalanced = "RefBldgSmallOfficeNew2004_USA_FL_Miami.Intl.AP.722020_TMY3.csv"

In [ ]:
def load_file(fname, max_load):
    df = pd.read_csv(fname, skiprows=range(1,49)).set_index(pd.date_range(start='1/1/2018 00:00', periods=8760, freq='H')).drop(labels=['Date/Time'], axis=1)
    df = df / (3600 * 1000)

    d = {}
    for x in list(df.columns):
        d[x] = x.replace('[J]', '[kWh]')

    df.rename(columns=d, inplace=True)
    df.rename(columns=lambda x : x.strip(), inplace=True)
    
    df['Total Heating Energy [kWh]']  = df[['PSZ-AC:1:Air System Total Heating Energy [kWh](Hourly)', 
                                            'PSZ-AC:2:Air System Total Heating Energy [kWh](Hourly)',
                                            'PSZ-AC:3:Air System Total Heating Energy [kWh](Hourly)',
                                            'PSZ-AC:4:Air System Total Heating Energy [kWh](Hourly)',
                                            'PSZ-AC:5:Air System Total Heating Energy [kWh](Hourly)']].sum(axis=1)

    df['Total Cooling Energy [kWh]']  = df[['PSZ-AC:1:Air System Total Cooling Energy [kWh](Hourly)', 
                                            'PSZ-AC:2:Air System Total Cooling Energy [kWh](Hourly)',
                                            'PSZ-AC:3:Air System Total Cooling Energy [kWh](Hourly)',
                                            'PSZ-AC:4:Air System Total Cooling Energy [kWh](Hourly)',
                                            'PSZ-AC:5:Air System Total Cooling Energy [kWh](Hourly)']].sum(axis=1)
    
    df['Balance'] = df['Total Cooling Energy [kWh]'] - df['Total Heating Energy [kWh]']
    df['Load [W]'] = df['Balance'] * 1000
    df['Load [W]'] = df['Load [W]'] * (max_load / df['Load [W]'].abs().max())
    
    return df

In [ ]:
df_b = load_file(balanced, 5000)

In [ ]:
df_b.plot(y=['Load [W]'])

In [ ]:
df_i = load_file(imbalanced, 4000)

In [ ]:
df_i.plot(y=['Load [W]'])

In [ ]:
d = {"Balance [kWh]": [df_b['Balance'].sum(), df_i['Balance'].sum()],
     "Total Cooling Energy [kWh]": [df_b['Total Cooling Energy [kWh]'].sum(), df_i['Total Cooling Energy [kWh]'].sum()],
 "Total Heating Energy [kWh]": [df_b['Total Heating Energy [kWh]'].sum(), df_i['Total Heating Energy [kWh]'].sum()]}

df = pd.DataFrame(data=d, index=['Balanced', 'Imbalanced'])
df

In [ ]:
df_b['Mass Flow Rate [kg/s]'] = 0.1
df_i['Mass Flow Rate [kg/s]'] = 0.1

In [ ]:
df_b.to_csv('balanced.csv', columns=['Load [W]', 'Mass Flow Rate [kg/s]'])
df_i.to_csv('imbalanced.csv', columns=['Load [W]', 'Mass Flow Rate [kg/s]'])

In [ ]:
df_energy_monthly = pd.DataFrame(data={'Balanced': df_b['Balance'], 
                                       'Imbalanced': df_i['Balance']})

df_energy_monthly = df_energy_monthly.resample('M').sum() / 76.2

fig = plt.figure(figsize=(7,5))
ax = fig.add_subplot(111)

bar_width = 0.35
index = np.arange(len(df_energy_monthly['Balanced']))

ax.bar(index, df_energy_monthly['Balanced'], bar_width, label="Balanced")
ax.bar(index + bar_width, df_energy_monthly['Imbalanced'], bar_width, label="Imbalanced")

ax.set_title('Net GHE Energy \n (+) Cooling (-) Heating')
ax.set_ylabel('kWh/m')
ax.set_xticks(index + bar_width / 2)
ax.set_xticklabels(('Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'))

plt.grid()
plt.legend()
plt.savefig('Balanced-Imbalanced_Energy_Balance.pdf', bbox_inches='tight')